# Peer-graded Assignment: Week 3 Part 1

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## 2. Get the data from Wikipedia page and parse the HTML/XML codes

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'xml')

## 3. Extract raw table and cleaning it

In [3]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

## 4. Combine postal codes with more than one neighbor

In [4]:
unique_p = set(Postcode)
print('Number of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Number of unique Postal codes: 84


## 5. Create Pandas DataFrame 

In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M2L,North York,"Silver Hills, York Mills"
1,M2R,North York,Willowdale West
2,M2J,North York,Henry Farm
3,M5M,North York,Bedford Park
4,M3C,North York,Flemingdon Park
5,M1M,Scarborough,"Cliffcrest, Cliffside"
6,M1J,Scarborough,Scarborough Village
7,M9P,Etobicoke,Westmount
8,M4J,East York,East Toronto
9,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"


## 6. Print the number of rows in the dataframes

In [6]:
print ("Number of rows in the dataframe = ", len(df_toronto.index))

Number of rows in the dataframe =  84


## 7. Export the dataset to .csv file

In [7]:
df_toronto.to_csv('downloads/Toronto_Postcodes_1.csv')